In [4]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

print("POPULATING DimGDP")

print("\nReading silver_gdp...")
df_gdp = spark.table("silver_gdp")

print(f"Silver GDP records: {df_gdp.count():,}")

print("\nSchema:")
df_gdp.printSchema()
print("\nSample silver data:")
df_gdp.show(5, truncate=False)

df_dim_gdp = df_gdp.select(
    col("year").alias("Year"),
    col("country").alias("CountryName"),
    col("gdp_value_usd").alias("GDP")
).filter(col("year").between(2019, 2024))

df_dim_gdp = df_dim_gdp.withColumn(
    "GDPKey",
    row_number().over(Window.orderBy("Year"))
)

df_dim_gdp = df_dim_gdp.select(
    "GDPKey", 
    "Year", 
    "CountryName", 
    "GDP"
)

print("\nTransformed data:")
df_dim_gdp.show(10, truncate=False)

print("\nWriting to Lakehouse table: DimGDP")
df_dim_gdp.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("DimGDP")

print(f"DimGDP created with {df_dim_gdp.count()} records!")

StatementMeta(, d4f8e96e-548f-41af-803b-6c35324dc49a, 6, Finished, Available, Finished)

POPULATING DimGDP

Reading silver_gdp...
Silver GDP records: 25

Schema:
root
 |-- year: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- indicator: string (nullable = true)
 |-- gdp_value_usd: double (nullable = true)
 |-- gdp_value_billions: double (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)
 |-- year_over_year_growth_pct: double (nullable = true)


Sample silver data:
+----+-------------+-----------------+---------------+------------------+-------------------------+-------------------------+
|year|country      |indicator        |gdp_value_usd  |gdp_value_billions|ingestion_timestamp      |year_over_year_growth_pct|
+----+-------------+-----------------+---------------+------------------+-------------------------+-------------------------+
|2000|United States|GDP (current US$)|1.0250947997E13|10250.95          |2026-01-05 09:23:05.06554|NULL                     |
|2001|United States|GDP (current US$)|1.0581929774E13|10581.93          |